## Setup & Configuration <a id="setup"></a>

First, let's load environment variables and import common dependencies.

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
import os
import httpx
from typing import Annotated
from pathlib import Path
from dotenv import load_dotenv
from pydantic import Field

# Load environment variables
local_env_path = Path.cwd() / ".env"
parent_env_path = Path.cwd().parent.parent.parent / ".env"
load_dotenv(dotenv_path=local_env_path)
load_dotenv(dotenv_path=parent_env_path)

print("Environment loaded successfully!")
print(f"Project Endpoint: {os.getenv('AZURE_AI_PROJECT_ENDPOINT', 'Not set')}")
print(f"Model Deployment: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME', 'Not set')}")

### Helper Functions

Define reusable helper functions for weather data.

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the current weather for a given location using OpenWeatherMap API."""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    if not api_key:
        return f"Error: OPENWEATHER_API_KEY not found in environment variables."
    
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        response = httpx.get(url, timeout=10.0)
        response.raise_for_status()
        data = response.json()
        
        temp = data["main"]["temp"]
        feels_like = data["main"]["feels_like"]
        description = data["weather"][0]["description"]
        humidity = data["main"]["humidity"]
        
        return f"The weather in {location} is {description} with a temperature of {temp}°C (feels like {feels_like}°C) and {humidity}% humidity."
    except httpx.HTTPStatusError as e:
        if e.response.status_code == 404:
            return f"Error: Location '{location}' not found."
        return f"Error fetching weather data: {e}"
    except Exception as e:
        return f"Error: {str(e)}"

print("Helper functions defined!")

---

## 1. Basic Agent <a id="basic"></a>

The simplest way to create an agent using `AzureAIClient`. Demonstrates both streaming and non-streaming responses.

In [ ]:
from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential

async def basic_agent_demo():
    print("=== Basic Azure AI Agent Example ===")
    
    # Non-streaming example
    print("\n--- Non-streaming Response ---")
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="BasicWeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Toronto?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")
    
    # Streaming example
    print("\n--- Streaming Response ---")
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="BasicWeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Mexico City?"
        print(f"User: {query}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print()

await basic_agent_demo()

---

## 2. Using Latest Version <a id="latest-version"></a>

Demonstrates how to reuse the latest version of an existing agent instead of creating a new agent version on each instantiation.

In [ ]:
async def latest_version_demo():
    print("=== Using Latest Agent Version ===")
    
    async with AzureCliCredential() as credential:
        # First call creates a new agent
        async with (
            AzureAIClient(
                async_credential=credential,
            ).create_agent(
                name="MyWeatherAgent",
                instructions="You are a helpful weather agent.",
                tools=get_weather,
            ) as agent,
        ):
            query = "What's the weather like in Toronto?"
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result}")

        # Second call reuses latest version
        print("\n--- Reusing Latest Version ---")
        async with (
            AzureAIClient(
                async_credential=credential,
                use_latest_version=True,  # This parameter reuses the latest agent version
            ).create_agent(
                name="MyWeatherAgent",
                instructions="You are a helpful weather agent.",
                tools=get_weather,
            ) as agent,
        ):
            query = "What's the weather like in Mexico City?"
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result}")

await latest_version_demo()

---

## 3. Agent-to-Agent (A2A) <a id="a2a"></a>

Demonstrates Agent-to-Agent communication using the A2A protocol.

**Prerequisites:** Requires `A2A_PROJECT_CONNECTION_ID` environment variable.

In [ ]:
async def a2a_demo():
    print("=== Agent-to-Agent (A2A) Example ===")
    
    if not os.getenv("A2A_PROJECT_CONNECTION_ID"):
        print("⚠️  Skipping: A2A_PROJECT_CONNECTION_ID environment variable not set")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyA2AAgent",
            instructions="""You are a helpful assistant that can communicate with other agents.
            Use the A2A tool when you need to interact with other agents to complete tasks
            or gather information from specialized agents.""",
            tools={
                "type": "a2a_preview",
                "project_connection_id": os.environ["A2A_PROJECT_CONNECTION_ID"],
            },
        ) as agent,
    ):
        query = "What can the secondary agent do?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await a2a_demo()

---

## 4. Azure AI Search <a id="azure-search"></a>

Search through indexed data using Azure AI Search.

**Prerequisites:** `AI_SEARCH_PROJECT_CONNECTION_ID` and `AI_SEARCH_INDEX_NAME` environment variables.

In [ ]:
async def azure_search_demo():
    print("=== Azure AI Search Example ===")
    
    if not os.getenv("AI_SEARCH_PROJECT_CONNECTION_ID") or not os.getenv("AI_SEARCH_INDEX_NAME"):
        print("⚠️  Skipping: AI_SEARCH_PROJECT_CONNECTION_ID and AI_SEARCH_INDEX_NAME required")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MySearchAgent",
            instructions="""You are a helpful assistant. You must always provide citations for
            answers using the tool and render them as: `[message_idx:search_idx†source]`.""",
            tools={
                "type": "azure_ai_search",
                "azure_ai_search": {
                    "indexes": [
                        {
                            "project_connection_id": os.environ["AI_SEARCH_PROJECT_CONNECTION_ID"],
                            "index_name": os.environ["AI_SEARCH_INDEX_NAME"],
                            "query_type": "simple",
                        }
                    ]
                },
            },
        ) as agent,
    ):
        query = "Tell me about insurance options"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await azure_search_demo()

---

## 5. Bing Custom Search <a id="bing-custom"></a>

Search custom search instances using Bing Custom Search.

**Prerequisites:** `BING_CUSTOM_SEARCH_PROJECT_CONNECTION_ID` and `BING_CUSTOM_SEARCH_INSTANCE_NAME`.

In [ ]:
async def bing_custom_search_demo():
    print("=== Bing Custom Search Example ===")
    
    if not os.getenv("BING_CUSTOM_SEARCH_PROJECT_CONNECTION_ID"):
        print("⚠️  Skipping: BING_CUSTOM_SEARCH_PROJECT_CONNECTION_ID required")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyCustomSearchAgent",
            instructions="""You are a helpful agent that can use Bing Custom Search tools to assist users.
            Use the available Bing Custom Search tools to answer questions and perform tasks.""",
            tools={
                "type": "bing_custom_search_preview",
                "bing_custom_search_preview": {
                    "search_configurations": [
                        {
                            "project_connection_id": os.environ["BING_CUSTOM_SEARCH_PROJECT_CONNECTION_ID"],
                            "instance_name": os.environ["BING_CUSTOM_SEARCH_INSTANCE_NAME"],
                        }
                    ]
                },
            },
        ) as agent,
    ):
        query = "Tell me more about foundry agent service"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await bing_custom_search_demo()

---

## 6. Bing Grounding <a id="bing-grounding"></a>

Search the web for current information using Bing Grounding.

**Prerequisites:** `BING_PROJECT_CONNECTION_ID` environment variable.

In [ ]:
async def bing_grounding_demo():
    print("=== Bing Grounding Example ===")
    
    if not os.getenv("BING_PROJECT_CONNECTION_ID"):
        print("⚠️  Skipping: BING_PROJECT_CONNECTION_ID required")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyBingGroundingAgent",
            instructions="""You are a helpful assistant that can search the web for current information.
            Use the Bing search tool to find up-to-date information and provide accurate, well-sourced answers.
            Always cite your sources when possible.""",
            tools={
                "type": "bing_grounding",
                "bing_grounding": {
                    "search_configurations": [
                        {
                            "project_connection_id": os.environ["BING_PROJECT_CONNECTION_ID"],
                        }
                    ]
                },
            },
        ) as agent,
    ):
        query = "What is today's date and weather in Toronto?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await bing_grounding_demo()

---

## 7. Code Interpreter <a id="code-interpreter"></a>

Execute Python code for mathematical problem solving using the `HostedCodeInterpreterTool`.

In [ ]:
from agent_framework import ChatResponse, HostedCodeInterpreterTool
from openai.types.responses.response import Response as OpenAIResponse
from openai.types.responses.response_code_interpreter_tool_call import ResponseCodeInterpreterToolCall

async def code_interpreter_demo():
    print("=== Code Interpreter Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyCodeInterpreterAgent",
            instructions="You are a helpful assistant that can write and execute Python code to solve problems.",
            tools=HostedCodeInterpreterTool(),
        ) as agent,
    ):
        query = "Use code to get the factorial of 100?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

        # Extract generated code if available
        if (
            isinstance(result.raw_representation, ChatResponse)
            and isinstance(result.raw_representation.raw_representation, OpenAIResponse)
            and len(result.raw_representation.raw_representation.output) > 0
        ):
            code_interpreter_item = next(
                (
                    item
                    for item in result.raw_representation.raw_representation.output
                    if isinstance(item, ResponseCodeInterpreterToolCall)
                ),
                None,
            )

            if code_interpreter_item is not None:
                generated_code = code_interpreter_item.code
                print(f"\nGenerated code:\n{generated_code}")

await code_interpreter_demo()

---

## 8. Existing Agent <a id="existing-agent"></a>

Work with a pre-existing agent by providing agent name and version.

In [ ]:
from agent_framework import ChatAgent
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition

async def existing_agent_demo():
    print("=== Existing Agent Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as project_client,
    ):
        # Create a test agent
        azure_ai_agent = await project_client.agents.create_version(
            agent_name="MyNewTestAgent",
            definition=PromptAgentDefinition(
                model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
                instructions="End each response with [END].",
            ),
        )

        chat_client = AzureAIClient(
            project_client=project_client,
            agent_name=azure_ai_agent.name,
            agent_version=azure_ai_agent.version,
        )

        try:
            async with ChatAgent(
                chat_client=chat_client,
            ) as agent:
                query = "How are you?"
                print(f"User: {query}")
                result = await agent.run(query)
                print(f"Agent: {result}")
        finally:
            # Cleanup
            await project_client.agents.delete_version(
                agent_name=azure_ai_agent.name, agent_version=azure_ai_agent.version
            )

await existing_agent_demo()

---

## 9. Existing Conversation <a id="existing-conversation"></a>

Use an existing conversation created on the service side.

In [ ]:
async def existing_conversation_demo():
    print("=== Existing Conversation Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as project_client,
    ):
        # Create a conversation
        openai_client = await project_client.get_openai_client()
        conversation = await openai_client.conversations.create()
        conversation_id = conversation.id
        print(f"Created conversation: {conversation_id}")

        async with AzureAIClient(
            project_client=project_client,
            conversation_id=conversation_id,
        ).create_agent(
            name="BasicAgent",
            instructions="You are a helpful agent.",
            tools=get_weather,
        ) as agent:
            query = "What's the weather like in Guadalajara?"
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result.text}")

            query = "What was my last question?"
            print(f"\nUser: {query}")
            result = await agent.run(query)
            print(f"Agent: {result.text}")

await existing_conversation_demo()

---

## 10. Explicit Settings <a id="explicit-settings"></a>

Create an agent with explicitly configured settings instead of environment defaults.

In [ ]:
from random import randint

def get_mock_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get mock weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

async def explicit_settings_demo():
    print("=== Explicit Settings Example ===")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIClient(
                project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
                model_deployment_name=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
                async_credential=credential,
                agent_name="WeatherAgent",
            ),
            instructions="You are a helpful weather agent.",
            tools=get_mock_weather,
        ) as agent,
    ):
        query = "What's the weather like in New York?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await explicit_settings_demo()

---

## 11. File Search <a id="file-search"></a>

Upload files and enable agents to search through documents using `HostedFileSearchTool`.

In [ ]:
from agent_framework import HostedFileSearchTool, HostedVectorStoreContent
from azure.ai.agents.aio import AgentsClient
from azure.ai.agents.models import FileInfo, VectorStore

async def file_search_demo():
    print("=== File Search Example ===")
    
    file = None
    vector_store = None

    async with (
        AzureCliCredential() as credential,
        AgentsClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as agents_client,
        AzureAIClient(async_credential=credential) as client,
    ):
        try:
            # Upload file
            pdf_file_path = Path.cwd().parent / "resources" / "employees.pdf"
            if not pdf_file_path.exists():
                print(f"⚠️  Skipping: File not found at {pdf_file_path}")
                return
                
            print(f"Uploading file: {pdf_file_path}")
            file = await agents_client.files.upload_and_poll(file_path=str(pdf_file_path), purpose="assistants")
            print(f"Uploaded file ID: {file.id}")

            vector_store = await agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="my_vectorstore")
            print(f"Created vector store ID: {vector_store.id}")

            # Create file search tool
            file_search_tool = HostedFileSearchTool(inputs=[HostedVectorStoreContent(vector_store_id=vector_store.id)])

            async with ChatAgent(
                chat_client=client,
                name="EmployeeSearchAgent",
                instructions="You are a helpful assistant that can search through uploaded employee files.",
                tools=file_search_tool,
            ) as agent:
                queries = [
                    "Who is the youngest employee?",
                    "Who works in sales?",
                ]
                for query in queries:
                    print(f"\nUser: {query}")
                    response = await agent.run(query)
                    print(f"Agent: {response.text}")
        finally:
            # Cleanup
            if vector_store:
                await agents_client.vector_stores.delete(vector_store.id)
            if file:
                await agents_client.files.delete(file.id)

await file_search_demo()

---

## 12. Hosted MCP <a id="hosted-mcp"></a>

Integrate hosted Model Context Protocol (MCP) tools with Azure AI Agent.

In [ ]:
from agent_framework import HostedMCPTool

async def hosted_mcp_demo():
    print("=== Hosted MCP Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyLearnDocsAgent",
            instructions="You are a helpful assistant that can help with Microsoft documentation questions.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
                approval_mode="never_require",
            ),
        ) as agent,
    ):
        query = "How to create an Azure storage account using az cli?"
        print(f"User: {query}")
        result = await agent.run(query, store=False)
        print(f"Agent: {result}")

await hosted_mcp_demo()

---

## 13. Image Generation <a id="image-generation"></a>

Generate images based on text prompts using the `ImageGenTool`.

In [ ]:
from agent_framework import DataContent
from azure.ai.projects.models import ImageGenTool
import aiofiles

async def image_generation_demo():
    print("=== Image Generation Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="ImageGenAgent",
            instructions="Generate images based on user requirements.",
            tools=[ImageGenTool(quality="low", size="1024x1024")],
        ) as agent,
    ):
        query = "Generate an image of Microsoft logo."
        print(f"User: {query}")
        result = await agent.run(
            query,
            additional_chat_options={
                "extra_headers": {"x-ms-oai-image-generation-deployment": "gpt-image-1"},
            },
        )
        print(f"Agent: {result}")

        # Save image if available
        image_data = [
            content
            for content in result.messages[0].contents
            if isinstance(content, DataContent) and content.media_type == "image/png"
        ]
        if image_data and image_data[0]:
            filename = "microsoft.png"
            file_path = Path.cwd() / filename
            async with aiofiles.open(file_path, "wb") as f:
                await f.write(image_data[0].get_data_bytes())
            print(f"Image saved to: {file_path}")

await image_generation_demo()

---

## 14. Response Format (Structured Outputs) <a id="response-format"></a>

Use Pydantic models to enforce specific response schemas.

In [ ]:
from pydantic import BaseModel, ConfigDict

class ReleaseBrief(BaseModel):
    feature: str
    benefit: str
    launch_date: str
    model_config = ConfigDict(extra="forbid")

async def response_format_demo():
    print("=== Response Format (Structured Outputs) Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="ProductMarketerAgent",
            instructions="Return launch briefs as structured JSON.",
        ) as agent,
    ):
        query = "Draft a launch brief for the Contoso Note app."
        print(f"User: {query}")
        result = await agent.run(query, response_format=ReleaseBrief)

        if isinstance(result.value, ReleaseBrief):
            release_brief = result.value
            print("Agent:")
            print(f"Feature: {release_brief.feature}")
            print(f"Benefit: {release_brief.benefit}")
            print(f"Launch date: {release_brief.launch_date}")

await response_format_demo()

---

## 15. Search Context - Agentic Mode <a id="search-agentic"></a>

Use Azure AI Search with agentic mode for multi-hop reasoning.

**Prerequisites:** Multiple environment variables required (see code).

In [ ]:
from agent_framework_aisearch import AzureAISearchContextProvider
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

async def search_context_agentic_demo():
    print("=== Search Context - Agentic Mode ===")
    
    required_vars = [
        "AZURE_SEARCH_ENDPOINT",
        "AZURE_SEARCH_INDEX_NAME",
        "AZURE_SEARCH_KNOWLEDGE_BASE_NAME",
        "AZURE_OPENAI_RESOURCE_URL",
    ]
    
    if not all(os.getenv(var) for var in required_vars):
        print(f"⚠️  Skipping: Required variables: {', '.join(required_vars)}")
        return
    
    search_provider = AzureAISearchContextProvider(
        endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
        index_name=os.environ["AZURE_SEARCH_INDEX_NAME"],
        credential=DefaultAzureCredential(),
        mode="agentic",
        azure_ai_project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
        azure_openai_resource_url=os.environ["AZURE_OPENAI_RESOURCE_URL"],
        model_deployment_name=os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        knowledge_base_name=os.environ["AZURE_SEARCH_KNOWLEDGE_BASE_NAME"],
        top_k=3,
    )

    async with (
        search_provider,
        AzureAIAgentClient(
            project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
            model_deployment_name=os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            async_credential=DefaultAzureCredential(),
        ) as client,
        ChatAgent(
            chat_client=client,
            name="SearchAgent",
            instructions="You are a helpful assistant with advanced reasoning capabilities.",
            context_providers=[search_provider],
        ) as agent,
    ):
        query = "What information is available in the knowledge base?"
        print(f"User: {query}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print()

await search_context_agentic_demo()

---

## 16. Search Context - Semantic Mode <a id="search-semantic"></a>

Use Azure AI Search with semantic mode for fast hybrid search.

In [ ]:
async def search_context_semantic_demo():
    print("=== Search Context - Semantic Mode ===")
    
    if not os.getenv("AZURE_SEARCH_ENDPOINT") or not os.getenv("AZURE_SEARCH_INDEX_NAME"):
        print("⚠️  Skipping: AZURE_SEARCH_ENDPOINT and AZURE_SEARCH_INDEX_NAME required")
        return
    
    search_provider = AzureAISearchContextProvider(
        endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
        index_name=os.environ["AZURE_SEARCH_INDEX_NAME"],
        credential=DefaultAzureCredential(),
        mode="semantic",
        top_k=3,
    )

    async with (
        search_provider,
        AzureAIAgentClient(
            project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
            model_deployment_name=os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            async_credential=DefaultAzureCredential(),
        ) as client,
        ChatAgent(
            chat_client=client,
            name="SearchAgent",
            instructions="You are a helpful assistant. Use the provided context from the knowledge base.",
            context_providers=[search_provider],
        ) as agent,
    ):
        query = "What information is available in the knowledge base?"
        print(f"User: {query}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print()

await search_context_semantic_demo()

---

## 17. Thread Management <a id="thread-management"></a>

Demonstrate thread persistence and conversation context management.

In [ ]:
async def thread_management_demo():
    print("=== Thread Management Example ===")
    
    # Example with thread persistence in-memory
    print("\n--- Thread Persistence (In-Memory) ---")
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="BasicWeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        thread = agent.get_new_thread()

        query1 = "What's the weather like in Mexico City?"
        print(f"User: {query1}")
        result1 = await agent.run(query1, thread=thread, store=False)
        print(f"Agent: {result1.text}")

        query2 = "How about Guadalajara?"
        print(f"\nUser: {query2}")
        result2 = await agent.run(query2, thread=thread, store=False)
        print(f"Agent: {result2.text}")

        query3 = "Which of the cities I asked about has better weather?"
        print(f"\nUser: {query3}")
        result3 = await agent.run(query3, thread=thread, store=False)
        print(f"Agent: {result3.text}")

await thread_management_demo()

---

## 18. Web Search <a id="web-search"></a>

Perform web searches using the `HostedWebSearchTool`.

In [ ]:
from agent_framework import HostedWebSearchTool

async def web_search_demo():
    print("=== Web Search Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="WebsearchAgent",
            instructions="You are a helpful assistant that can search the web",
            tools=[HostedWebSearchTool()],
        ) as agent,
    ):
        query = "What's the weather today in Toronto?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await web_search_demo()

---

## Summary

This notebook demonstrated 18 different patterns for working with Azure AI agents:

1. ✅ Basic agent with streaming/non-streaming
2. ✅ Using latest agent version
3. ✅ Agent-to-Agent communication
4. ✅ Azure AI Search integration
5. ✅ Bing Custom Search
6. ✅ Bing Grounding for web search
7. ✅ Code Interpreter for Python execution
8. ✅ Working with existing agents
9. ✅ Existing conversation management
10. ✅ Explicit configuration settings
11. ✅ File search with vector stores
12. ✅ Hosted MCP tool integration
13. ✅ Image generation
14. ✅ Structured outputs with response format
15. ✅ Search context - Agentic mode
16. ✅ Search context - Semantic mode
17. ✅ Thread management for conversations
18. ✅ Web search capabilities

Each example showcases different capabilities and integration patterns available with Microsoft Agent Framework and Azure AI.